# Toronto Dwellings Analysis Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions to create a Panel dashboard.

In [86]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [87]:
# Initialize the Panel Extensions (for Plotly)
import panel as pn
pn.extension("plotly")

In [88]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token(map_box_api)

# Import Data

In [89]:
# Import the CSVs to Pandas DataFrames
file_path = Path("Data/toronto_neighbourhoods_census_data.csv")
to_data = pd.read_csv(file_path, index_col="year")

file_path = Path("Data/toronto_neighbourhoods_coordinates.csv")
df_neighbourhood_locations = pd.read_csv(file_path)

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

### Global available data

In [90]:
# Getting the data from the top 10 expensive neighbourhoods
to_topten = to_data.reset_index(drop=True).groupby("neighbourhood").mean().sort_values(by='average_house_value', ascending=False).head(10)

# Calculate the mean number of dwelling types units per year
to_avg_nh_values = to_data.reset_index(drop=True).groupby("neighbourhood").mean().reset_index()

# Calculate the average monthly shelter costs for owned and rented dwellings
to_data_owned = to_data.groupby("year")[['shelter_costs_owned']].mean().sort_index()
to_data_rental = to_data.groupby("year")[['shelter_costs_rented']].mean().sort_index()

to_data_owned_rental = pd.DataFrame()

to_data_owned_rental["shelter_costs_owned"] = to_data_owned["shelter_costs_owned"]
to_data_owned_rental["shelter_costs_rented"] = to_data_rental["shelter_costs_rented"]

# Calculate the average house value per year
to_data_avg_house = to_data.groupby("year")[['average_house_value']].mean().sort_index()

#Average value by Neighbourhood
to_data_avg_nb = to_data.groupby(["year", "neighbourhood"])[['average_house_value']].mean().sort_index().reset_index()

#Distinct neighbourhoods
to_neighbourhoods = to_data["neighbourhood"].reset_index(drop=True).drop_duplicates()

#Fetch the data of all dwelling types per year
to_data_hvplot = to_data.reset_index()

#Dwelling types per year
to_data_year = to_data.groupby("year").sum().drop(columns=['average_house_value', 'shelter_costs_owned', 'shelter_costs_rented'])

# Create a bar chart per year to show the number of dwelling types
data_2001 = to_data_year.loc[to_data_year.index == 2001].melt(value_name="Dwelling Type Units", var_name='2001')
data_2006 = to_data_year.loc[to_data_year.index == 2006].melt(value_name="Dwelling Type Units", var_name='2006')
data_2011 = to_data_year.loc[to_data_year.index == 2011].melt(value_name="Dwelling Type Units", var_name='2011')
data_2016 = to_data_year.loc[to_data_year.index == 2016].melt(value_name="Dwelling Type Units", var_name='2016')


### Panel Visualization Functions

In [91]:
# Define Panel visualization functions
def neighbourhood_map():
    
    to_nh_loc_data = pd.concat(
    [df_neighbourhood_locations.set_index('neighbourhood'), to_avg_nh_values.set_index('neighbourhood')], axis="columns", join="inner").reset_index()

    population_plot = px.scatter_mapbox(
    to_nh_loc_data,
    hover_name="neighbourhood",
    hover_data=["shelter_costs_owned", "other_house"], #just adding these random columns to showcase the array possibility of hover_data
    lat="lat",
    lon="lon",
    size="average_house_value",
    color="average_house_value",
    color_continuous_scale=px.colors.cyclical.IceFire,
    title="Average House Values in Toronto",
    zoom=10,
    width=1000,
    height=600)
    
    return population_plot


def create_bar_chart(data, title, xlabel, ylabel, color):
    
    bar_plot = px.bar(
        data,
        x=xlabel,
        y=ylabel,
        title=title,
        height=500,
        width=500
    )
    
    return bar_plot.update_traces(marker_color=color)
    


def create_line_chart(data, title, xlabel, ylabel, color):
    
    return data.hvplot(label=title, xlabel=xlabel, ylabel=ylabel).opts(line_color=color)


def choose_nh(neighbourhood):   
    
    #set new df
    to_data_avg_distinct_nb = to_data_avg_nb.loc[to_data_avg_nb['neighbourhood'] == neighbourhood].set_index('year')['average_house_value']
    
    return create_line_chart(to_data_avg_distinct_nb, "Neighbourhood: " + neighbourhood, "Year", "Avg. House Value", "blue")
    
    
def choose_nh_hvplot(neighbourhood):   
    
    #set new df
    to_data_dwelling = to_data_hvplot.loc[to_data_hvplot['neighbourhood'] == neighbourhood].drop(columns=["neighbourhood"])
    
    to_dwelling_unpivoted = pd.melt(to_data_dwelling, id_vars=['year'], value_vars=['single_detached_house','apartment_five_storeys_plus','movable_dwelling','semi_detached_house','row_house','duplex','apartment_five_storeys_less','other_house'])
            
    return to_dwelling_unpivoted.set_index(['year', 'variable']).hvplot.bar(label="Neighbourhood: " + neighbourhood, xlabel="Year", ylabel="Dwelling Type Units", stacked=False,height=500, rot=90)

    
def top_most_expensive_neighbourhoods():
    
    return to_topten['average_house_value'].hvplot.bar(label="Top 10 Most Expensive Neighbourhoods in Toronto", xlabel="Neighbourhood", ylabel="Avg House Value", stacked=False,height=500, rot=90)



## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [92]:
# Create a Title for the Dashboard
# YOUR CODE HERE!

# Define a welcome text
welcome_text = "## This dashboard presents a visual analysis of historical house values, dwelling types and costs in Toronto, ON according to census data from 2001 to 2016. You can navigate through the tabs above to explore more details."

# Create a tab layout for the dashboard
neighbourhood_tab = pn.Column(welcome_text, neighbourhood_map)

## Create two rows
decade1_row = pn.Row(create_bar_chart(data_2001, 'Dwelling Types in Toronto in 2001', '2001', 'Dwelling Type Units', 'blue'),
                    create_bar_chart(data_2006, 'Dwelling Types in Toronto in 2006', '2006', 'Dwelling Type Units', 'red'))
decade2_row = pn.Row(create_bar_chart(data_2011, 'Dwelling Types in Toronto in 2011', '2011', 'Dwelling Type Units', 'orange'),
                    create_bar_chart(data_2016, 'Dwelling Types in Toronto in 2016', '2016', 'Dwelling Type Units', 'green'))

# Add rows as columns
yearly_market_tab = pn.Column(
"## Yearly Dwelling Types", decade1_row, decade2_row
)

## Column
shelter_house_tab = pn.Column(
    create_line_chart(to_data_owned_rental["shelter_costs_owned"], "Average Monthly Shelter Cost for Owned Dwellings in Toronto", "Year", "Avg Monthly Shelter Costs", "orange"),
    create_line_chart(to_data_owned_rental["shelter_costs_rented"], "Average Monthly Shelter Cost for Rented Dwellings in Toronto", "Year", "Avg Monthly Shelter Costs", "red"),
    create_line_chart(to_data_avg_house, "Average House Value in Toronto", "Year", "Avg. House Value", "blue")
)


##Interactive charts
from panel.interact import interact

nh_analysis_tab = pn.Column(
    interact(choose_nh, neighbourhood=to_neighbourhoods),
    interact(choose_nh_hvplot, neighbourhood=to_neighbourhoods)
)


# Create the main dashboard
dashboard = pn.Tabs(
    ("Welcome", neighbourhood_tab),
    ("Yearly Market Analysis", yearly_market_tab),
    ("Shelter Costs Vs. House Value", shelter_house_tab),
    ("Neighbourhood Analysis", nh_analysis_tab),
    ("Top Expensive Neighbourhoods", top_most_expensive_neighbourhoods))



## Serve the Panel Dashboard

In [93]:
dashboard.servable()

Tabs
    [0] Column
        [0] Markdown(str)
        [1] Column
            [0] Column()
            [1] Row
                [0] Plotly(Figure, name='interactive10252')
    [1] Column
        [0] Markdown(str)
        [1] Row
            [0] Plotly(Figure)
            [1] Plotly(Figure)
        [2] Row
            [0] Plotly(Figure)
            [1] Plotly(Figure)
    [2] Column
        [0] HoloViews(Curve)
        [1] HoloViews(Curve)
        [2] HoloViews(Curve)
    [3] Column
        [0] Column
            [0] Column
                [0] Select(name='neighbourhood', options=['Agincourt North', ...], value='Agincourt North')
            [1] Row
                [0] HoloViews(Curve, name='interactive10639')
        [1] Column
            [0] Column
                [0] Select(name='neighbourhood', options=['Agincourt North', ...], value='Agincourt North')
            [1] Row
                [0] HoloViews(Bars, name='interactive10766')
    [4] Column
        [0] Column()
        [1] Row
            [0] HoloViews(Bars, name='interactive10852')